In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Shawn\\Desktop\\TextSummarizer\\app\\research'

In [3]:
os.chdir("../")

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfiguration:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from src.constants import *
from src.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath: Path = CONFIG_FILE_PATH,
                 params_filepath: Path = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfiguration:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        return DataIngestionConfiguration(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [8]:
import os
import urllib.request as request
import zipfile
from src.logging import logger
from src.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfiguration):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file to the unzip directory
        Returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-12-24 16:52:04,793: INFO: common: Reading config\config.yaml file]
[2024-12-24 16:52:04,795: INFO: common: Reading params.yaml file]
[2024-12-24 16:52:04,798: INFO: common: Directory created: artifacts]
[2024-12-24 16:52:04,801: INFO: common: Directory created: artifacts/data_ingestion]
[2024-12-24 16:52:07,787: INFO: 2390706015: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 780A:338308:382DFC:54582E:676A7634
Accept-Ranges: bytes
Date: Tue, 24 Dec 2024 08:52:05 GMT
Via: 1.1 varnish
X-Served-By: cache-qpg1270-QPG
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1735030325.1